> **Note:** In most sessions you will be solving exercises posed in a Jupyter notebook that looks like this one. Because you are cloning a Github repository that only we can push to, you should **NEVER EDIT** any of the files you pull from Github. Instead, what you should do, is either make a new notebook and write your solutions in there, or **make a copy of this notebook and save it somewhere else** on your computer, not inside the `sds` folder that you cloned, so you can write your answers in there. If you edit the notebook you pulled from Github, those edits (possible your solutions to the exercises) may be overwritten and lost the next time you pull from Github. This is important, so don't hesitate to ask if it is unclear.

# Exercise Set 13: Model building process and model selection

*Morning, August 21, 2018*

In this Exercise Set 13 we will investigate how to build machine learning models using a formalize pipeline from preprocessed (i.e. tidy) data to a model.

We import our standard stuff. Notice that we are not interested in seeing the convergence warning in scikit-learn so we suppress them for now.

In [1]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import seaborn as sns

## Model validation

In what follows we will regard the "train" data for two purposes. First we are interested in performing a model selection. Then with the selected model we estimate/train it on all the training data. 


> **Ex. 13.1.0:** Begin by reloading the housing dataset from Ex. 12.2.0 using the code below. 

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

cal_house = fetch_california_housing()    
X = pd.DataFrame(data=cal_house['data'], 
                 columns=cal_house['feature_names'])\
             .iloc[:,:-2]
y = cal_house['target']

> **Ex. 13.1.1:** Make a for loop with 10 iterations where you:
1. Split the input data into, train (also know as development) and test where the test sample should be one third. (Set a new random state for each iteration of the loop, so each iteration makes a different split).
2. Further split the training (aka development) data into to even sized bins; the first data is for training models and the other is for validating them. (Therefore these data sets are often called training and validation)
3. Train a linear regression model with sub-training data. Compute the RMSE for out-of-sample predictions for both the test data  and the validation data. Save the RMSE.

> You should now have a 10x2 DataFrame with 10 RMSE from both the test data set and the train data set. Compute descriptive statistics of RMSE for the out-of-sample predictions on test and validation data. Are they simular?    
>   They hopefuly are pretty simular. This shows us, that we can split the train data, and use this to fit the model. 

>> *Hint*: you can reuse any code used to solve exercises 12.2.X.

In [3]:
# [Answer to Ex. 13.1.1]
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from math import sqrt

rmses_val = []
rmses_test = []


for i in range(0,10):
    X_dev, X_test, y_dev, y_test = train_test_split(X, y, test_size=1/3, random_state=i)
    X_train, X_val, y_train, y_val = train_test_split(X_dev, y_dev, test_size=0.5, random_state=i)
    
    pipe_lr = make_pipeline(StandardScaler(), LinearRegression())
    
    pipe_lr.fit(X_train, y_train)
    y_pred = pipe_lr.predict(X_val)
    y_pred_test = pipe_lr.predict(X_test)
    rmses_val.append(sqrt(mse(y_pred, y_val)))
    rmses_test.append(sqrt(mse(y_pred_test, y_test)))

rmses_val
rmses_test

done = pd.DataFrame({"RMSE Val": rmses_val, "RMSE Test": rmses_test})
done

,RMSE Val,RMSE Test
0,0.779470,0.799316
1,0.797080,0.784875
2,0.781067,0.813145
3,0.792573,0.794923
4,0.777620,0.786323
5,3.899375,2.504750
6,1.229534,2.178635
7,0.801040,0.790696
8,0.797176,0.773043
9,0.783332,0.793141


## Model building

> **Ex. 13.1.2:** Construct a model building pipeline which 

> 1. adds polynomial features of degree 3 without bias;
> 1. scales the features to mean zero and unit std. 
> 1. estimates a Lasso model

>> *Hint:* a modelling pipeline can be constructed with `make_pipeline` from `sklearn.pipeline`.

In [4]:
# [Answer to Ex. 13.1.2]
from sklearn.preprocessing import PolynomialFeatures


pipe_lassoCV = make_pipeline(PolynomialFeatures(degree=3, include_bias=False),
                                     StandardScaler(),
                                     Lasso(alpha=_, random_state=1))

## Cross validation
In machine learning, we have two types of parameters: those that are learned from
the training data, for example, the weights in logistic regression, and the parameters
of a learning algorithm that are optimized separately. The latter are the tuning
parameters, also called *hyperparameters*, of a model, for example, the regularization
parameter in logistic regression or the depth parameter of a decision tree.
  
   
When we want to optimize over both normal parameters and hyperparameteres we do this using nested loops (two-layered cross validation). In outer loop we vary the hyperparameters, and then in the inner loop we do cross validation for the model with the specific selection of hyperparameters. This way we can find the model, with the lowest mean MSE. 

> **Ex. 13.1.3:**
Run a Lasso regression using the Pipeline from `Ex 13.1.2`. In the outer loop searching through the lambdas specified below. 
In the inner loop make *5 fold cross validation* on the selected model and store the average MSE for each fold. Which lambda gives the lowest test MSE?


> ```python 
lambdas =  np.logspace(-4, 4, 12)
```

>> *Hint:* `KFold` in `sklearn.model_selection` may be useful.

In [5]:
# [Answer to Ex. 13.1.3]
from sklearn.model_selection import KFold

kfolds = KFold(n_splits=5)
#folds = kfolds.split(X_dev, y_dev)
lambdas = np.logspace(-4, 4, 12)

# outer loop: lambdas
mseCV = []

for lambda_ in lambdas:
    # inner loop: folds
    mseCV_ = []    
    for train_idx, val_idx in kfolds.split(X_dev, y_dev):       # for each split we make, make a model 
        # train model and compute MSE on test fold
        pipe_lassoCV = make_pipeline(PolynomialFeatures(degree=3, include_bias=False),
                                     StandardScaler(),
                                     Lasso(alpha=lambda_, random_state=1))            
        X_train, y_train, = X_dev.iloc[train_idx], y_dev[train_idx]
        X_val, y_val = X_dev.iloc[val_idx], y_dev[val_idx] 
        pipe_lassoCV.fit(X_train, y_train)        #first we fit model on training data
        mseCV_.append(mse(pipe_lassoCV.predict(X_val), y_val))   #then we predict the validation set 
        
    # store result    
    mseCV.append(mseCV_) 
    
# convert to DataFrame
lambdaCV = pd.DataFrame(mseCV, index=lambdas)
#lambdaCV.head()
#lambdaCV.mean(axis=1)
lambdaCV # leftmost column is the value of lambda, the other columns are the respective kfold values

,0,1,2,3,4
0.000100,0.525493,0.662894,0.506985,0.594421,0.843436
0.000534,0.532407,0.772071,0.506748,0.550583,11.240687
0.002848,0.543194,0.545168,0.504095,0.524556,2.833196
0.015199,0.580298,0.571591,0.558977,0.599879,0.587422
0.081113,0.645711,0.664023,0.644697,0.688057,0.669344
0.432876,0.822063,0.900591,0.876513,0.942351,0.874513
2.310130,1.232622,1.333391,1.314193,1.400686,1.335353
12.328467,1.232622,1.333391,1.314193,1.400686,1.335353
65.793322,1.232622,1.333391,1.314193,1.400686,1.335353
351.119173,1.232622,1.333391,1.314193,1.400686,1.335353


> **Ex. 13.1.4:** __Automated Cross Validation in one dimension__  
Now we want to repeat exercise 13.1.3 in a more automated fasion. 
When you are doing cross validation with one hyperparameter, you can automate the process by using `validation_curve` from `sklearn.model_selection`. Use this function to search through the values of lambda, and find the value of lambda, which give the lowest test error.  

> check if you got the same output for the manual implementation (Ex. 13.1.3) and the automated implementation (Ex. 13.1.4) 

> BONUS: Plot the average MSE-test and MSE-train against the different values of lambda. (*Hint*: Use logarithmic axes, and lambda as index)

In [6]:
# [Answer to Ex. 13.1.4]
from sklearn.model_selection import validation_curve

train_scores, test_scores = \
    validation_curve(estimator=pipe_lassoCV,
                     X=X_dev,
                     y=y_dev,
                     param_name='lasso__alpha',
                     param_range=lambdas,
                     scoring='neg_mean_squared_error',                 
                     cv=5)

mse_score = pd.DataFrame({'Train':-train_scores.mean(axis=1),
                          'Validation':-test_scores.mean(axis=1),
                          'lambda':lambdas})\
              .set_index('lambda')   
print(mse_score.Validation.nsmallest(1)) # 0.5796 is the mean of all of those kfold columns

lambda
0.015199    0.579633
Name: Validation, dtype: float64


When you have *more than one* hyperparameter, you will want to fit the model to all the possible combinations of hyperparameters. This is done in an approch called `Grid Search`, which is implementet in `sklearn.model_selection` as `GridSearchCV`

> **Ex. 13.1.5:** To get to know `Grid Search` we want to implement in one dimension. Using `GridSearchCV` implement the Lasso, with the same lambdas as before (`lambdas =  np.logspace(-4, 4, 12)`), 10-fold CV and (negative) mean squared error as the scoring variable. Which value of Lambda gives the lowest test error? 

In [8]:
# [Answer to Ex. 13.1.5]
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

gs = GridSearchCV(estimator=pipe_lassoCV, 
                  param_grid={'lasso__alpha':np.logspace(-4,4,12)},  # l1 ratio
                  scoring='neg_mean_squared_error', # gridsearch is a maximizer, so since we want to minimize, take a neg. MSE
                  n_jobs=4, # makes your computation faster
                  iid=False,
                  cv=5)


In [9]:
gs.fit(X_dev, y_dev)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('polynomialfeatures',
                                        PolynomialFeatures(degree=3,
                                                           include_bias=False,
                                                           interaction_only=False,
                                                           order='C')),
                                       ('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('lasso',
                                        Lasso(alpha=10000.0, copy_X=True,
                                              fit_intercept=True, max_iter=1000,
                                              normalize=...
       

In [11]:
gs.best_params_

{'lasso__alpha': 0.01519911082952933}

> **Ex. 13.1.6 BONUS** Expand the Lasso pipe from the last excercise with a Principal Component Analisys (PCA), and expand the Grid Search to searching in two dimensions (both along the values of lambda and the values of principal components (n_components)). Is `n_components` a hyperparameter? Which hyperparameters does the Grid Search select as the best?

> *About PCA*: Principal Component Analysis, or PCA, is a dimensionality-reduction method that is often used to reduce the dimensionality of large data sets, by transforming a large set of variables into a smaller one that still contains most of the information in the large set. Read more about it online e.g. [here](https://towardsdatascience.com/a-step-by-step-explanation-of-principal-component-analysis-b836fb9c97e2)

> NB. This might take a while to calculate. 

In [16]:
# [Answer to Ex. 13.1.6]

> **Ex. 13.1.7 BONUS** repeat the previous now with RandomizedSearchCV with 20 iterations.

In [18]:
# [Answer to Ex. 13.1.7]



> **Ex. 13.1.8 BONUS** read about nested cross validation. How might we implement this in answer 13.1.6?


In [20]:
# [Answer to Ex. 13.1.8]